In [1]:
import requests
import geopandas as gpd
import pandas as pd
from sodapy import Socrata
import altair as alt
import numpy as np






# Garden Suites

In [2]:
# garden and garage suite data direct from website
#reference: https://data.edmonton.ca/Urban-Planning-Economy/General-Building-Permits-Garage-and-Garden-Suites/wkbv-nqd4/about_data



client = Socrata("data.edmonton.ca", None)
'''client = Socrata(data.edmonton.ca,
                  MyAppToken,
                  username="a.almusa@utoronto.ca",
                  password="ccccc")'''
#First 2000 results, returned as JSON from API / converted to Python list of
#dictionaries by sodapy.

results = client.get("wkbv-nqd4", limit=1000000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

results_df



,job_description,legal_description,count,location,construction_value,address,neighbourhood_numberr,units_added,latitude,zoning,...,year,job_category,longitude,building_type,row_id,permit_date,neighbourhood,issue_date,month_number,work_type
0,To construct a Garden Suite.,Plan 1822264 Blk 27 Lot 9,1,"{'latitude': '53.60394972493329', 'human_addre...",77375,6218G - HAMPTON GRAY AVENUE NW,3111,1,53.60394972493329,GLDF,...,2024,"Single, Semi-detached & Rowhousing",-113.50928323687212,Backyard House (110),2-492437524,2024-02-16T00:00:00.000,GRIESBACH,2024-02-16T00:00:00.000,2,(01) Building - New
1,To construct a two-Storey Garden Suite (main f...,Plan 1822264 Blk 27 Lot 8,1,"{'latitude': '53.603949897593864', 'human_addr...",77375,6220G - HAMPTON GRAY AVENUE NW,3111,1,53.603949897593864,GLDF,...,2024,"Single, Semi-detached & Rowhousing",-113.50945445138474,Backyard House (110),2-492431806,2024-02-16T00:00:00.000,GRIESBACH,2024-02-16T00:00:00.000,2,(01) Building - New
2,To construct a Garden Suite with balcony and c...,Plan 3159HW Blk 15B Lot 20,1,"{'latitude': '53.561452857371', 'human_address...",62500,11440G - 112 AVENUE NW,1170,1,53.561452857371,RS,...,2024,"Single, Semi-detached & Rowhousing",-113.51931783290478,Backyard House (110),2-481943803,2024-02-15T00:00:00.000,PRINCE RUPERT,2024-02-15T00:00:00.000,2,(01) Building - New
3,To construct a two storey Garden Suite (one Su...,Plan RN43 Blk 5 Lot 10,1,"{'latitude': '53.566723388248064', 'human_addr...",120000,11531G - 101 STREET NW,1230,1,53.566723388248064,RS,...,2024,"Single, Semi-detached & Rowhousing",-113.49319542856117,Backyard House (110),2-489176001,2024-02-13T00:00:00.000,SPRUCE AVENUE,2024-02-13T00:00:00.000,2,(01) Building - New
4,To construct a Garden Suite (Garage on Main Fl...,Plan 5430HW Blk 14 Lot 39,1,"{'latitude': '53.578397306394', 'human_address...",107625,12342G - 131 STREET NW,3410,1,53.578397306394,RS,...,2024,"Single, Semi-detached & Rowhousing",-113.55174325900033,Backyard House (110),2-469461882,2024-02-12T00:00:00.000,SHERBROOKE,2024-02-12T00:00:00.000,2,(01) Building - New
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
823,To construct a Detached Garage with a Garage S...,Plan 6046HW Blk 3 Lot 5,1,"{'latitude': '53.50068704014026', 'human_addre...",114000,8744 - 64 AVENUE NW,6010,1,53.50068704014026,NaN,...,2009,House Combination,-113.46613096806992,Single Detached House (110),2-85677027,2009-08-24T00:00:00.000,ARGYLL,2009-08-24T00:00:00.000,8,(01) New
824,To construct an addition to a Single Detached ...,Plan 4065AE Blk 1 Lots 15-16,1,"{'latitude': '53.564079437959435', 'human_addr...",25000,6306 - 111 AVENUE NW,2330,0,53.564079437959435,RF1,...,2009,Other Miscellaneous Building,-113.43430111117793,Single Detached House (110),1-83897191,2009-08-12T00:00:00.000,HIGHLANDS,2009-08-12T00:00:00.000,8,(02) Addition
825,To construct an Accessory Building (Detached G...,Plan 2725Q Blk 3 Lot 18,1,"{'latitude': '53.528119614838374', 'human_addr...",9936,9233A - 96 STREET NW,6040,0,53.528119614838374,NaN,...,2009,Accessory Building Combination,-113.47466525450548,Detached Garage (010),0-84817208,2009-06-09T00:00:00.000,BONNIE DOON,2009-06-09T00:00:00.000,6,(01) New
826,To construct an accessory Building (Detached G...,Plan 8438ET Blk 6 Lot A,1,"{'latitude': '53.529130309259735', 'human_addr...",5000,9312 - 93 AVENUE NW,6040,1,53.529130309259735,RF3,...,2009,Other Miscellaneous Building,-113.47039810962804,Detached Garage (010),1-78275327,2009-05-28T00:00:00.000,BONNIE DOON,2009-05-28T00:00:00.000,5,(01) Building - New


In [3]:
#results_df.columns
#results_df['count'].max()

In [4]:
#dropping some columns
col_drop = ['legal_description', 
            'count', 'construction_value', 
            'floor_area', 'job_category', 
            'row_id', 
            'zoning', 
            'neighbourhood_numberr', 
            'location',
            'units_added', 
            'neighbourhood', 
            'month_number', 
            'work_type']
results_df = results_df.drop(col_drop, axis=1)

In [5]:
results_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 828 entries, 0 to 827
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   job_description  828 non-null    object
 1   address          828 non-null    object
 2   latitude         826 non-null    object
 3   year             828 non-null    object
 4   longitude        826 non-null    object
 5   building_type    826 non-null    object
 6   permit_date      828 non-null    object
 7   issue_date       828 non-null    object
dtypes: object(8)
memory usage: 51.9+ KB


In [6]:
#convert permit date t date and time
results_df['permit_date'] = pd.to_datetime(results_df['permit_date'])

# Convert 'issue_date' to datetime
results_df['issue_date'] = pd.to_datetime(results_df['issue_date'])

# convert year to date time
results_df['year'] = pd.to_datetime(results_df['year'], format='%Y').dt.year


In [7]:
# renaming columns:
results_df.rename(columns={'issue_date': 'permit_issued_date'}, inplace=True)

standard_columns_order = ['year','permit_issued_date', 'permit_date', 'address', 'longitude', 'latitude', 'job_description' ]

results_df = results_df[standard_columns_order]
#add a date_completed column 
results_df['date_completed'] = np.nan

results_df.to_csv('edmonton_gs.csv')
results_df

,year,permit_issued_date,permit_date,address,longitude,latitude,job_description,date_completed
0,2024,2024-02-16,2024-02-16,6218G - HAMPTON GRAY AVENUE NW,-113.50928323687212,53.60394972493329,To construct a Garden Suite.,NaN
1,2024,2024-02-16,2024-02-16,6220G - HAMPTON GRAY AVENUE NW,-113.50945445138474,53.603949897593864,To construct a two-Storey Garden Suite (main f...,NaN
2,2024,2024-02-15,2024-02-15,11440G - 112 AVENUE NW,-113.51931783290478,53.561452857371,To construct a Garden Suite with balcony and c...,NaN
3,2024,2024-02-13,2024-02-13,11531G - 101 STREET NW,-113.49319542856117,53.566723388248064,To construct a two storey Garden Suite (one Su...,NaN
4,2024,2024-02-12,2024-02-12,12342G - 131 STREET NW,-113.55174325900033,53.578397306394,To construct a Garden Suite (Garage on Main Fl...,NaN
...,...,...,...,...,...,...,...,...
823,2009,2009-08-24,2009-08-24,8744 - 64 AVENUE NW,-113.46613096806992,53.50068704014026,To construct a Detached Garage with a Garage S...,NaN
824,2009,2009-08-12,2009-08-12,6306 - 111 AVENUE NW,-113.43430111117793,53.564079437959435,To construct an addition to a Single Detached ...,NaN
825,2009,2009-06-09,2009-06-09,9233A - 96 STREET NW,-113.47466525450548,53.528119614838374,To construct an Accessory Building (Detached G...,NaN
826,2009,2009-05-28,2009-05-28,9312 - 93 AVENUE NW,-113.47039810962804,53.529130309259735,To construct an accessory Building (Detached G...,NaN


## Visualizing GS Data

In [8]:
buildings_per_year = results_df['year'].value_counts().reset_index()
buildings_per_year.columns = ['year', 'Number of Buildings']
buildings_per_year_sorted = buildings_per_year.sort_values('year')

In [9]:
buildings_per_year_sorted

,year,Number of Buildings
13,2009,10
11,2010,11
14,2011,9
15,2012,6
12,2013,10
10,2014,21
8,2015,33
7,2016,60
5,2017,65
6,2018,64


In [10]:
# Creating a bar chart
bars = alt.Chart(buildings_per_year_sorted).mark_bar().encode(
    x=alt.X('year:O', axis=alt.Axis(title='Year')),
    y=alt.Y('Number of Buildings:Q', axis=alt.Axis(title='Number of Suites'))
)

# Creating text marks to display the number on top of each bar
text = bars.mark_text(
    align='center',
    baseline='bottom',
    dy=-5  # Nudging the text a bit above the bars
).encode(
    text='Number of Buildings:Q'
)

# Layering the text marks on top of the bars
chart = (bars + text).properties(
    title='Number of Garden and Garage Suites Permits per Year',
    width=600,
    height=400
)

chart

alt.LayerChart(...)

In [11]:
# Basic Statistical Analysis
mean_buildings = buildings_per_year_sorted['Number of Buildings'].mean()
median_buildings = buildings_per_year_sorted['Number of Buildings'].median()
mode_buildings = buildings_per_year_sorted['Number of Buildings'].mode()[0]
std_dev_buildings = buildings_per_year_sorted['Number of Buildings'].std()

# Calculating minimum, maximum, and total count
min_buildings = buildings_per_year_sorted['Number of Buildings'].min()
max_buildings = buildings_per_year_sorted['Number of Buildings'].max()
total_buildings = buildings_per_year_sorted['Number of Buildings'].sum()

# Identifying the years with minimum and maximum buildings
year_min_buildings = buildings_per_year_sorted[buildings_per_year_sorted['Number of Buildings'] == min_buildings]['year'].values
year_max_buildings = buildings_per_year_sorted[buildings_per_year_sorted['Number of Buildings'] == max_buildings]['year'].values

# Printing the results
print(f"Mean (Average): {mean_buildings}")
print(f"Median: {median_buildings}")
print(f"Mode: {mode_buildings}")
print(f"Standard Deviation: {std_dev_buildings}")
print(f"Minimum Number of Buildings: {min_buildings} in year(s) {year_min_buildings}")
print(f"Maximum Number of Buildings: {max_buildings} in year(s) {year_max_buildings}")
print(f"Total Number of Buildings: {total_buildings}")

Mean (Average): 51.75
Median: 46.5
Mode: 10
Standard Deviation: 42.24926034855522
Minimum Number of Buildings: 6 in year(s) [2012]
Maximum Number of Buildings: 128 in year(s) [2021]
Total Number of Buildings: 828


In [12]:
# issued_permits_per_year.info()

In [13]:
'''# aggregating data by year
issued_permits_per_year['year'] = pd.to_datetime(issued_permits_per_year['year'], format='%Y').dt.year

issued_permits_per_year = results_df.groupby('year')['address'].count().reset_index()
issued_permits_per_year['year'] = pd.to_datetime(issued_permits_per_year['year'], format='%Y').dt.year

print(issued_permits_per_year)'''

"# aggregating data by year\nissued_permits_per_year['year'] = pd.to_datetime(issued_permits_per_year['year'], format='%Y').dt.year\n\nissued_permits_per_year = results_df.groupby('year')['address'].count().reset_index()\nissued_permits_per_year['year'] = pd.to_datetime(issued_permits_per_year['year'], format='%Y').dt.year\n\nprint(issued_permits_per_year)"

In [14]:
'''# Creating a bar chart
bars = alt.Chart(issued_permits_per_year).mark_bar().encode(
    x=alt.X('year', axis=alt.Axis(title='Year')),
    y=alt.Y('address', axis=alt.Axis(title='Number of Suites'))
)

# Creating text marks to display the number on top of each bar
text = bars.mark_text(
    align='center',
    baseline='bottom',
    dy=-5  # Nudging the text a bit above the bars
).encode(
    text='address'
)

# Layering the text marks on top of the bars
chart = (bars + text).properties(
    title='Number of Garden and Garage Suites Permits per Year',
    width=600,
    height=400
)

chart'''

"# Creating a bar chart\nbars = alt.Chart(issued_permits_per_year).mark_bar().encode(\n    x=alt.X('year', axis=alt.Axis(title='Year')),\n    y=alt.Y('address', axis=alt.Axis(title='Number of Suites'))\n)\n\n# Creating text marks to display the number on top of each bar\ntext = bars.mark_text(\n    align='center',\n    baseline='bottom',\n    dy=-5  # Nudging the text a bit above the bars\n).encode(\n    text='address'\n)\n\n# Layering the text marks on top of the bars\nchart = (bars + text).properties(\n    title='Number of Garden and Garage Suites Permits per Year',\n    width=600,\n    height=400\n)\n\nchart"

# Secondary Suites

In [15]:
# ref: https://dev.socrata.com/foundry/data.edmonton.ca/uz9h-ceya
# ref: https://data.edmonton.ca/Urban-Planning-Economy/Secondary-Suites-Completed-Permits-/q3qs-7g3d


# Secondary Suites data direct from website

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.edmonton.ca", None)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.edmonton.ca,
#                  MyAppToken,
#                  username="user@example.com",
#                  password="AFakePassword")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results_ss = client.get("uz9h-ceya", limit=50000)

# Convert to pandas DataFrame
results_df_ss = pd.DataFrame.from_records(results_ss)

results_df_ss

,job_number,permit_issued_date,permit_type,permit_description,statscan_building_type,address,neighbourhood,neighbourhood_number,zoning,longitude,latitude,location,:@computed_region_7ccj_gre3,:@computed_region_ecxu_fw7u,:@computed_region_izdr_ja4x,:@computed_region_5jki_au6x,:@computed_region_mnf4_kaez,:@computed_region_eq8d_jmrp
0,000663526-001,1998-08-12T00:00:00.000,Development Permit,To operate a combined three unit Apartment Hou...,Apartments (310),10513 - 74 AVENUE NW,Queen Alexandra,5330,RF3,-113.5015521472843,53.51007541772635,"{'latitude': '53.51007541772635', 'longitude':...",13,10,6,5,11,252
1,000691014-001,1999-01-27T00:00:00.000,Major Development Permit,To continue to operate a 3 unit Apartment Hous...,Semi-Detached House (210),"BSMTA, 9850 - 81 AVENUE NW",Ritchie,6610,RF3,-113.48530555050084,53.51732846910685,"{'latitude': '53.51732846910685', 'longitude':...",156,66,7,5,3,290
2,000704855-004,2014-07-08T00:00:00.000,7. Other Misc. Building Permits,To develop a Secondary Suite in the basement o...,Single Detached House (110),"BSMT, 4011 - 28A STREET NW",Larkspur,6390,RS,-113.38344434913097,53.47615656729781,"{'latitude': '53.47615656729781', 'longitude':...",76,135,7,7,4,204
3,000957222-005,2012-02-03T00:00:00.000,7. Other Misc. Building Permits,To construct a Secondary Suite in a Single Hou...,Single Detached House (110),"2, 11053 - 108 STREET NW",Central McDougall,1030,RS,-113.50606843145002,53.55823464674358,"{'latitude': '53.55823464674358', 'longitude':...",181,220,3,5,9,84
4,024968667-001,2003-01-27T00:00:00.000,7. Other Misc. Building Permits,"Develop suite in basement, approved 879020",Single Detached House (110),7606 - 110 STREET NW,McKernan,5290,RS,-113.5154754109622,53.51256108345719,"{'latitude': '53.51256108345719', 'longitude':...",59,18,6,5,11,231
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9799,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-113.48312933314335,53.5735668911347,"{'latitude': '53.5735668911347', 'longitude': ...",42,155,4,3,10,3
9800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-113.4464854729,53.51487963789361,"{'latitude': '53.51487963789361', 'longitude':...",374,74,7,5,3,194
9801,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-113.50771780459782,53.49787506343797,"{'latitude': '53.49787506343797', 'longitude':...",50,19,6,5,2,278
9802,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-113.59340854861337,53.5482736798302,"{'latitude': '53.5482736798302', 'longitude': ...",30,87,3,1,1,59


In [16]:
results_df_ss.columns

Index(['job_number', 'permit_issued_date', 'permit_type', 'permit_description',
       'statscan_building_type', 'address', 'neighbourhood',
       'neighbourhood_number', 'zoning', 'longitude', 'latitude', 'location',
       ':@computed_region_7ccj_gre3', ':@computed_region_ecxu_fw7u',
       ':@computed_region_izdr_ja4x', ':@computed_region_5jki_au6x',
       ':@computed_region_mnf4_kaez', ':@computed_region_eq8d_jmrp'],
      dtype='object')

In [17]:
#dropping some columns
col_drop = ['job_number', 
            'neighbourhood_number', 
            'zoning',  
            'location',
            ':@computed_region_7ccj_gre3', ':@computed_region_ecxu_fw7u',
            ':@computed_region_izdr_ja4x', ':@computed_region_5jki_au6x',
            ':@computed_region_mnf4_kaez', ':@computed_region_eq8d_jmrp', 
            'neighbourhood', 
            'permit_type', 
            'statscan_building_type' ]

results_df_ss = results_df_ss.drop(col_drop, axis=1)




In [18]:
results_df_ss

,permit_issued_date,permit_description,address,longitude,latitude
0,1998-08-12T00:00:00.000,To operate a combined three unit Apartment Hou...,10513 - 74 AVENUE NW,-113.5015521472843,53.51007541772635
1,1999-01-27T00:00:00.000,To continue to operate a 3 unit Apartment Hous...,"BSMTA, 9850 - 81 AVENUE NW",-113.48530555050084,53.51732846910685
2,2014-07-08T00:00:00.000,To develop a Secondary Suite in the basement o...,"BSMT, 4011 - 28A STREET NW",-113.38344434913097,53.47615656729781
3,2012-02-03T00:00:00.000,To construct a Secondary Suite in a Single Hou...,"2, 11053 - 108 STREET NW",-113.50606843145002,53.55823464674358
4,2003-01-27T00:00:00.000,"Develop suite in basement, approved 879020",7606 - 110 STREET NW,-113.5154754109622,53.51256108345719
...,...,...,...,...,...
9799,NaN,NaN,NaN,-113.48312933314335,53.5735668911347
9800,NaN,NaN,NaN,-113.4464854729,53.51487963789361
9801,NaN,NaN,NaN,-113.50771780459782,53.49787506343797
9802,NaN,NaN,NaN,-113.59340854861337,53.5482736798302


In [19]:
filtered_df = results_df_ss[results_df_ss['address'].isna()]


filtered_df

# Edmonton City reply: Those 263 records you have identified were entered into the source systems with incorrect/invalid location data; effectively, 
# the non-spatial data for those permits is missing from the Open Data release.

,permit_issued_date,permit_description,address,longitude,latitude
9538,NaN,NaN,NaN,-113.5227149843108,53.511583391001366
9539,NaN,NaN,NaN,-113.5227149843108,53.511583391001366
9540,NaN,NaN,NaN,-113.5227149843108,53.511583391001366
9541,NaN,NaN,NaN,-113.47778931247281,53.56876550174336
9542,NaN,NaN,NaN,-113.46679928410737,53.51729854028845
...,...,...,...,...,...
9799,NaN,NaN,NaN,-113.48312933314335,53.5735668911347
9800,NaN,NaN,NaN,-113.4464854729,53.51487963789361
9801,NaN,NaN,NaN,-113.50771780459782,53.49787506343797
9802,NaN,NaN,NaN,-113.59340854861337,53.5482736798302


In [20]:
filtered_df2 = results_df_ss[results_df_ss['permit_issued_date'].isna()]
filtered_df2

# Edmonton City reply: The ~50 records that are more complete but missing permit issue date are incomplete in the source system.
# One approach for these might be to impute these missing values. Whether to exclude these records or the particular imputation strategy you choose 
# (if you choose that approach) would be up to you and would depend on the nature of your analysis.


,permit_issued_date,permit_description,address,longitude,latitude
10,NaN,To develop a Secondary Suite in the Basement o...,1805 - 35 AVENUE NW,-113.36967872945623,53.469234686432095
108,NaN,To construct a secondary suite in the basement...,"BSMT, 12836 - 88 STREET NW",-113.47669051850846,53.5871613643056
198,NaN,To develop a Secondary Suite in the basement o...,"BSMT, 1368 - 70 STREET SW",-113.449317563546,53.420653037753716
514,NaN,To develop a Secondary Suite (2 Bedrooms). Ins...,"BSMT, 3611 - 107 STREET NW",-113.50686709692903,53.470304672771924
739,NaN,To construct a Secondary Suite in a Single Det...,12102 - 91 STREET NW,-113.48163916487566,53.57495043184777
...,...,...,...,...,...
9799,NaN,NaN,NaN,-113.48312933314335,53.5735668911347
9800,NaN,NaN,NaN,-113.4464854729,53.51487963789361
9801,NaN,NaN,NaN,-113.50771780459782,53.49787506343797
9802,NaN,NaN,NaN,-113.59340854861337,53.5482736798302


In [21]:
#dropping the invalid location data rows / instances

results_df_ss = results_df_ss.dropna(subset=['address'])

#replacing the nan value in the "permit_issued_date" column, with the year 1900




In [22]:
results_df_ss['permit_issued_date'] = results_df_ss['permit_issued_date'].fillna('1900-01-01T00:00:00.000')


C:\Users\ajalm\AppData\Local\Temp\ipykernel_21280\3410824278.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_df_ss['permit_issued_date'] = results_df_ss['permit_issued_date'].fillna('1900-01-01T00:00:00.000')


In [23]:
results_df_ss['permit_issued_date'] = pd.to_datetime(results_df_ss['permit_issued_date'])



C:\Users\ajalm\AppData\Local\Temp\ipykernel_21280\2614466027.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_df_ss['permit_issued_date'] = pd.to_datetime(results_df_ss['permit_issued_date'])


## Visualizing SS

In [24]:
# creating a new column for the year
results_df_ss['year'] = pd.to_datetime(results_df_ss['permit_issued_date'], format='%Y').dt.year



C:\Users\ajalm\AppData\Local\Temp\ipykernel_21280\3819058755.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_df_ss['year'] = pd.to_datetime(results_df_ss['permit_issued_date'], format='%Y').dt.year


In [25]:
results_df_ss['year'] = results_df_ss['year'].astype(str)
results_df_ss['year']

C:\Users\ajalm\AppData\Local\Temp\ipykernel_21280\3921989003.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_df_ss['year'] = results_df_ss['year'].astype(str)


0       1998
1       1999
2       2014
3       2012
4       2003
        ... 
9533    1971
9534    1971
9535    1976
9536    1970
9537    1963
Name: year, Length: 9538, dtype: object

In [26]:
results_df_ss['year'].unique()

results_df_ss.columns

Index(['permit_issued_date', 'permit_description', 'address', 'longitude',
       'latitude', 'year'],
      dtype='object')

In [27]:
#renaming columns to standard naming convention 

results_df_ss.rename(columns={'permit_description': 'job_description'}, inplace=True)

#ordering columns
standard_columns_order = ['year','permit_issued_date', 'address',  'longitude', 'latitude', 'job_description' ]

results_df_ss = results_df_ss[standard_columns_order]

#adding dae_completed column
results_df_ss['date_completed'] = np.nan

results_df_ss.to_csv('edmonton_ss.csv')

results_df_ss

C:\Users\ajalm\AppData\Local\Temp\ipykernel_21280\1169793854.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_df_ss.rename(columns={'permit_description': 'job_description'}, inplace=True)


,year,permit_issued_date,address,longitude,latitude,job_description,date_completed
0,1998,1998-08-12,10513 - 74 AVENUE NW,-113.5015521472843,53.51007541772635,To operate a combined three unit Apartment Hou...,NaN
1,1999,1999-01-27,"BSMTA, 9850 - 81 AVENUE NW",-113.48530555050084,53.51732846910685,To continue to operate a 3 unit Apartment Hous...,NaN
2,2014,2014-07-08,"BSMT, 4011 - 28A STREET NW",-113.38344434913097,53.47615656729781,To develop a Secondary Suite in the basement o...,NaN
3,2012,2012-02-03,"2, 11053 - 108 STREET NW",-113.50606843145002,53.55823464674358,To construct a Secondary Suite in a Single Hou...,NaN
4,2003,2003-01-27,7606 - 110 STREET NW,-113.5154754109622,53.51256108345719,"Develop suite in basement, approved 879020",NaN
...,...,...,...,...,...,...,...
9533,1971,1971-06-24,"BSMT2, 10924 - 79 AVENUE NW",-113.5132570905658,53.51547930802583,Convert a one family dwelling to a two family ...,NaN
9534,1971,1971-06-21,"BSMT, 12346 - 90 STREET NW",-113.4801086184994,53.57969068809744,To convert an existing one family dwelling to ...,NaN
9535,1976,1976-03-25,"2FL, 8904 - CONNORS ROAD NW",-113.46384587433715,53.52843986057847,To convert one family dwelling to two family d...,NaN
9536,1970,1970-01-13,"BSMT, 7204 - 78 AVENUE NW",-113.4386478074487,53.514470305825924,To convert an existing one family dwelling to ...,NaN


In [28]:
results_df_ss.info()



<class 'pandas.core.frame.DataFrame'>
Index: 9538 entries, 0 to 9537
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   year                9538 non-null   object        
 1   permit_issued_date  9538 non-null   datetime64[ns]
 2   address             9538 non-null   object        
 3   longitude           9538 non-null   object        
 4   latitude            9538 non-null   object        
 5   job_description     9536 non-null   object        
 6   date_completed      0 non-null      float64       
dtypes: datetime64[ns](1), float64(1), object(5)
memory usage: 596.1+ KB


In [29]:


buildings_per_year_ss = results_df_ss['year'].value_counts().reset_index()
buildings_per_year_ss.columns = ['year', 'Number of Buildings']
buildings_per_year_sorted_ss = buildings_per_year_ss.sort_values('year')


In [30]:
# Creating a bar chart
bars = alt.Chart(buildings_per_year_sorted_ss).mark_bar().encode(
    x=alt.X('year:O', axis=alt.Axis(title='Year')),
    y=alt.Y('Number of Buildings:Q', axis=alt.Axis(title='Number of Suites'))
)

# Creating text marks to display the number on top of each bar
text = bars.mark_text(
    align='center',
    baseline='bottom',
    dy=-5  # Nudging the text a bit above the bars
).encode(
    text='Number of Buildings:Q'
)

# Layering the text marks on top of the bars
chart = (bars + text).properties(
    title='Number of Secondary Suites Permits per Year',
    width=600,
    height=400
)

chart

alt.LayerChart(...)

In [31]:
# Convert 'year' to integers before grouping
results_df_ss['year'] = pd.to_numeric(results_df_ss['year'], errors='coerce')

# Now apply the lambda function to group years, with conversion handled
results_df_ss['year_grouped'] = results_df_ss['year'].apply(lambda x: 'pre-2009' if x < 2009 else str(x))

# Follow the same steps as before to calculate counts and plot
buildings_per_year_grouped = results_df_ss['year_grouped'].value_counts().reset_index()
buildings_per_year_grouped.columns = ['year', 'Number of Buildings']

# Sorting, considering 'pre-2009' as a special case. This might need custom sorting logic if 'year' is mixed with strings and numbers
buildings_per_year_grouped['sort_key'] = buildings_per_year_grouped['year'].apply(lambda x: int(x) if x.isdigit() else 0)
buildings_per_year_grouped_sorted = buildings_per_year_grouped.sort_values(by='sort_key').drop('sort_key', axis=1)

# Explicitly define the order of categories, ensuring 'pre-2009' appears first
ordered_categories = ['pre-2009'] + sorted([year for year in buildings_per_year_grouped_sorted['year'] if year != 'pre-2009'], key=lambda x: int(x))

# Plotting with ordered x-axis categories
bars = alt.Chart(buildings_per_year_grouped_sorted).mark_bar().encode(
    x=alt.X('year:O', axis=alt.Axis(title='Year'), sort=ordered_categories),
    y=alt.Y('Number of Buildings:Q', axis=alt.Axis(title='Number of Suites'))
)

text = bars.mark_text(
    align='center',
    baseline='bottom',
    dy=-5  # Nudging the text a bit above the bars
).encode(
    text='Number of Buildings:Q'
)

chart = (bars + text).properties(
    title='Number of Secondary Suites Permits per Year',
    width=600,
    height=400
)

chart






alt.LayerChart(...)